In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch import optim
from torch.autograd import Variable
import math, time
import matplotlib.pyplot as plt
import copy
import os
import csv

In [2]:
def show_imgs(imgs, l1=4, l2=5, s1=6, s2=6, name=""):
    plt.rcParams['figure.figsize'] = (s1, s2)
    imgs = imgs.cpu().reshape([-1, 28, 28])
    g, ax = plt.subplots(l1, l2)
    for i in range(l1):
        for j in range(l2):
            a = i * l2 + j
            if (a >= imgs.shape[0]):
                break
            ax[i][j].imshow(imgs[a, :, :], cmap='gray')
            ax[i][j].set_xticks([])
            ax[i][j].set_yticks([])
    if name != "":
        plt.savefig(path + str(name) + ".png")
    #plt.show()
    plt.clf()

def show_loss(train_loss_s, test_loss_s):
    plt.rcParams['figure.figsize'] = (10, 10)
    plt.plot(train_loss_s, "o-", label='train loss')
    plt.plot(test_loss_s, "o-", label='train loss')
    plt.ylabel("loss")
    plt.xlabel("epoch")
    plt.legend(loc=1)
    plt.savefig(path + "loss.png")
    plt.clf()

def show_Acc(train_Acc_s, test_Acc_s):
    plt.rcParams['figure.figsize'] = (10, 10)
    plt.plot(train_Acc_s, "o-", label='train Acc')
    plt.plot(test_Acc_s, "o-", label='train Acc')
    plt.ylabel("Acc")
    plt.xlabel("epoch")
    plt.legend(loc=1)
    plt.savefig(path + "Acc.png")
    plt.clf()

In [3]:
class CMPS(nn.Module):
    def __init__ (self, Dmax, n, c, mydevice=torch.device('cpu')):
        super(CMPS,self).__init__()
        self.Dmax = Dmax
        self.n = n
        self.c = c
        self.bond_dims = [self.Dmax for i in range(n - 1)] + [1]
        self.tensors = []
        for i in range(self.n - 1):
            t = torch.randn(self.bond_dims[i - 1], 2, self.bond_dims[i], device=mydevice)
            t = Variable(t, requires_grad = True)
            self.tensors.append(t)
        t = torch.rand(self.bond_dims[self.n - 2], 2, self.bond_dims[self.n - 1], c, device=mydevice)
        t = Variable(t, requires_grad = True)
        self.tensors.append(t)
        self.normalize()
    
    def getNorm(self):
        result = torch.tensordot(self.tensors[0], self.tensors[0], dims=([1], [1]))
        for i in range(1,self.n - 1):
            result = torch.einsum("niol,ijk,ljm->nkom", result, self.tensors[i],self.tensors[i])
        result = torch.einsum("niol,ijkp,ljmp->nkomp", result, self.tensors[self.n - 1], self.tensors[self.n - 1])
        return result
    
    
    def normalize(self):
        result = torch.tensordot(self.tensors[0], self.tensors[0], dims=([1], [1]))
        res_max = result.max()
        tensor = self.tensors[0].detach()
        tensor = tensor / torch.sqrt(res_max)
        self.tensors[0] = Variable(tensor, requires_grad = True)
        result /= res_max
        for i in range(1,self.n - 1):
            result = torch.einsum("niol,ijk,ljm->nkom", result, self.tensors[i],self.tensors[i])
            res_max = result.max()
            tensor = self.tensors[i].detach()
            tensor = tensor / torch.sqrt(res_max)
            self.tensors[i] = Variable(tensor, requires_grad = True)
            result /= res_max           
        result = torch.einsum("niol,ijkp,ljmp->nkomp", result, self.tensors[self.n - 1], self.tensors[self.n - 1])
        tensor = self.tensors[self.n - 1].detach()
        for i in range(self.c):
            tensor[:,:,:,i] = tensor[:,:,:,i] / torch.sqrt(result[0,0,0,0,i])
        self.tensors[self.n - 1] = Variable(tensor, requires_grad = True)
    
    def forward(self,x):
        result = torch.einsum("ijk,lmn,bj,bm->bkn", self.tensors[0], self.tensors[0], x[:,0], x[:,0])
        for i in range(1, 784-1):
            result = torch.einsum("bil,ijk,lmn,bj,bm->bkn",result, self.tensors[i], self.tensors[i], x[:,i], x[:,i])
        result = torch.einsum("bil,ijkc,lmnc,bj,bm->bc",result, self.tensors[-1], self.tensors[-1], x[:,-1], x[:,-1])
        result = result/(self.getNorm().view(self.c))
        result = result * 1e300
        return result

In [4]:
def time_now():
    return time.strftime("%Y%m%d_%H%M%S", time.localtime())

def mapData2Spin(data):
    newData = torch.tensor(
        np.concatenate([
            np.cos(np.pi / 2 * data).reshape(len(data), n, 1),
            np.sin(np.pi / 2 * data).reshape(len(data), n, 1)
        ],
                       axis=2))
    return newData

In [5]:
def train(cmps,
          SpinData_train,
          train_label,
          SpinData_test,
          test_label,
          batch_size,
          epochs=5,
          learning_rate=0.001,
          mydevice=torch.device('cpu')):
    train_loss_s = []
    train_Acc_s = []
    test_loss_s = []
    test_Acc_s = []
    n = cmps.n
    start_time = time.time()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(cmps.tensors,lr=learning_rate)
    for epoch in range(epochs):
        train_loss = []
        train_Acc = []
        test_loss = []
        test_Acc = []
        maxnum = 0
        for batch_now in range(len(SpinData_train) // batch_size):
            SpinDatas = SpinData_train[(batch_now) *batch_size:(batch_now + 1) * batch_size]
            label = train_label[(batch_now) *batch_size:(batch_now + 1) * batch_size]
            out = cmps(SpinDatas)
            loss = criterion(out,label)
            loss.backward()
            optimizer.step()
            maxnum = max(maxnum,torch.max(out.data))
            pred_label = torch.max(out.data, 1).indices
            correct_cnt = (pred_label == label).sum()
            train_loss.append(loss.data)
            train_Acc.append(correct_cnt)
        for batch_now in range(len(SpinData_test) // batch_size):
            SpinDatas = SpinData_test[(batch_now) *batch_size:(batch_now + 1) * batch_size]
            label = test_label[(batch_now) *batch_size:(batch_now + 1) * batch_size]
            out = cmps(SpinDatas)
            loss = criterion(out,label)
            pred_label = torch.max(out.data, 1).indices
            correct_cnt = (pred_label == label).sum()
            test_loss.append(loss.data)
            test_Acc.append(correct_cnt)
        cmps.normalize()
        train_loss_s.append(np.mean(train_loss))
        train_Acc_s.append(np.sum(train_Acc)/len(SpinData_train))
        test_loss_s.append(np.mean(test_loss))
        test_Acc_s.append(np.sum(test_Acc)/len(SpinData_test))
        print(epoch + 1,train_loss_s[-1],train_Acc_s[-1],test_loss_s[-1],test_Acc_s[-1],maxnum)
        show_loss(train_loss_s, test_loss_s)
        show_Acc(train_Acc_s, test_Acc_s)

In [6]:
torch.set_default_tensor_type(torch.DoubleTensor)
plt.rcParams['figure.dpi'] = 300
mydevice = torch.device('cpu')
n = 784
m = 1000
tm = 500
Dmax = 5
c = 10
batch_size = 50
learning_rate = 0.005
path = time_now() + "_" + str(m) + "_" + str(batch_size) + "_" + str(
    tm) + "_" + str(Dmax) + "_" + str(learning_rate)
print(path)
os.makedirs(path)

20220724_211342_1000_50_500_5_0.005


In [7]:
path = "./" + path + "/"
raw_data = np.load("data_bin.npz")
train_data = raw_data['train_data']
train_data = torch.Tensor(train_data[:m, :])
show_imgs(train_data, 5, 15, 15, 5, "train_fig")
SpinData_train = mapData2Spin(train_data)
print(SpinData_train.shape)
test_data = raw_data['train_data']
test_data = torch.Tensor(test_data[:tm, :])
SpinData_test = mapData2Spin(test_data)
print(SpinData_test.shape)
train_label = torch.tensor(raw_data['train_label'][:m], dtype = torch.long)
test_label = torch.tensor(raw_data['test_label'][:m], dtype = torch.long)
print(train_label.shape)
print(test_label.shape)
torch.manual_seed(1)
np.random.seed(1)
SpinData_train = SpinData_train.to(mydevice)
SpinData_test = SpinData_test.to(mydevice)

cmps = CMPS(Dmax, n, c, mydevice)

torch.Size([1000, 784, 2])
torch.Size([500, 784, 2])
torch.Size([1000])
torch.Size([1000])


<Figure size 4500x1500 with 0 Axes>

In [8]:
train(cmps,SpinData_train,train_label,SpinData_test,test_label,batch_size,epochs = 2,learning_rate = 1e-3)

1 10933677.680152455 0.07 2.302585092994046 0.096 tensor(1.1018e+10)
2 2.3025850929940397 0.086 2.302585092994046 0.09 tensor(8.0480e-12)


<Figure size 3000x3000 with 0 Axes>

In [9]:
train(cmps,SpinData_train,train_label,SpinData_test,test_label,batch_size,epochs = 50,learning_rate = 1e-3)

1 2.3025850929940397 0.084 2.302585092994046 0.094 tensor(8.3650e-12)
2 2.3025850929940375 0.082 2.302585092994046 0.092 tensor(1.0964e-11)
3 2.3025850929940375 0.082 2.302585092994046 0.092 tensor(1.0964e-11)
4 2.3025850929940375 0.083 2.302585092994046 0.092 tensor(1.0964e-11)
5 2.3025850929940375 0.083 2.302585092994046 0.092 tensor(1.0964e-11)
6 2.3025850929940375 0.083 2.302585092994046 0.092 tensor(1.0964e-11)
7 2.3025850929940375 0.083 2.302585092994046 0.092 tensor(1.0964e-11)
8 2.3025850929940375 0.083 2.302585092994046 0.092 tensor(1.0964e-11)
9 2.3025850929940375 0.083 2.302585092994046 0.092 tensor(1.0964e-11)
10 2.3025850929940375 0.083 2.302585092994046 0.092 tensor(1.0964e-11)
11 2.3025850929940375 0.083 2.302585092994046 0.092 tensor(1.0964e-11)
12 2.3025850929940375 0.082 2.302585092994046 0.092 tensor(1.0964e-11)
13 2.3025850929940375 0.083 2.302585092994046 0.092 tensor(1.0964e-11)
14 2.3025850929940375 0.084 2.302585092994046 0.092 tensor(1.0964e-11)
15 2.3025850929

<Figure size 3000x3000 with 0 Axes>